In [14]:
import pandas as pd
import numpy as np
import ta 
from sklearn.linear_model import LogisticRegression
import math
from funcoes import *
pd.options.mode.chained_assignment = None


#dados foram baixados de https://www.cryptodatadownload.com/ 
#indicadores tecnicos de https://www.akademiabaru.com/doc/ARBMSV14_N1_P35_41.pdf
#indicadores são:  MACD, RSI, SO, MA, EMA, ROC e VT.


nrows = 1000
df = pd.read_csv("Binance_BTCUSDT_minute.csv",skiprows=1,nrows=nrows) #lendo somente as 10000 primeiras linhas para salvar processamento enquanto o codigo é desenvolvido
#lembrar de remover ultimas 74 rows 

calculate_indicators(df) #calcula os indicadores no df e retorna uma lista para ser usado no fit do modelo

df = df.iloc[25:] #é necessário ignorar as primeiras 25 linhas do dataframe pois alguns indicadores não possuem valores suficientes antes de si e, portanto, assumem valores Nan.
x = calc_x(df)

row_70 = math.floor(0.7*len(df))


#dataframe com 70% das linhas do banco de dados para treinar o modelo, determinar os coeficientes de regressão
df_train,x_train = df.iloc[:row_70],x[:row_70]
y_train = df_train.change

#dataframe com as linhas restantes para testar o modelo
df_test,x_test = df.iloc[row_70:],x[row_70:]
y_test = df_test.change


model = LogisticRegression(max_iter=1000)
model.fit(x_train,y_train)
coeficientes = list(model.coef_[0])
#coeficientes #MACD,SO,RSI,12_SMA,12_EMA,ROC -- > SO e ROC parecem ser pouco importantes. MACD e RSI parecem ser os mais importantes


lista_resultados = []
for i in x_test:
    resultado_row = calcula_p(i,coeficientes)
    lista_resultados.append(resultado_row)

acertos = 0
for i in range(0,len(lista_resultados)):
    if lista_resultados[i] >= 0.5 and df_test.iloc[i].change == 1:
        acertos += 1
    if lista_resultados[i] < 0.5 and df_test.iloc[i].change == 0:
        acertos += 1


acuracia = (acertos/len(df_train))*100
print(acuracia)

144201
53.21942986613646
